# xxx

> **Table of contents:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

*Imports and set magics*

In [1]:
import numpy as np
from scipy import optimize
import sympy as sm

# plotting
import matplotlib.pyplot as plt
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 14})

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

# Model description

**Write out the model in equations here.** 

Make sure you explain well the purpose of the model and comment so that other students who may not have seen it before can follow.  

**General setup:** Time is discrete and indexed by $t\in\{0,1,\dots\}$. In the model we have single infinitely-lived household. It is a closed economy and we ingnore the public sector. There are four markets: Goods market, labor market, rental market for capital goods and bond market. We assume perfect competition in all four markets

**Household:** Households supply labor exogenously, $N_{t}=1$, and earns a wage $w_{t}$. The return on saving is $r_{t+1}$. Utility is 

$$
\begin{aligned}
U & =\max_{\{C_{t}\}_{t=0}^{\infty}}\sum_{t=0}^{\infty}\beta^{t}\frac{C_{t}^{1-\sigma}}{1-\sigma},\beta\in(0,1),\sigma>0\\
 & \text{s.t.}\\
 & M_{t}=(1+r_{t})A_{t-1}+w_{t}N_{t}\\
 & A_{t}=M_{t}-C_{t}
\end{aligned}
$$

where $M_{t}$ is cash-on-hand and $A_{t}$ is end-of-period assets.

**Firms:** Firms rent capital $K_{t-1}$ at the rental rate $r_{t}^{K}$
and hires labor $L_{t}$ at the wage rate $w_{t}$.<br>
Firms have access to the production function
    
$$
\begin{aligned}
Y_{t} &= F(K_{t-1},L_{t}) \\ &=\Gamma_t(\alpha K_{t-1}^{-\theta}+(1-\alpha)L_{t}^{-\theta})^{\frac{1}{-\theta}},\,\,\,\theta>-1,\alpha\in(0,1),\Gamma_t>0
\end{aligned}
$$

Profits are

$$
\begin{aligned}
\Pi_{t}=Y_{t}-w_{t}L_{t}-r_{t}^{K}K_{t-1}
\end{aligned}
$$

**Equilibrium:**

1. Households maximize utility
2. Firms maximize profits
3. Labor market clear: $L_{t}=N_{t}=1$
4. Goods market clear: $Y_{t}=C_{t}+I_{t}$
5. Asset market clear: $A_{t}=K_{t}$ and $r_{t}=r_{t}^{k}-\delta$
6. Capital follows its law of motion: $K_{t}=(1-\delta)K_{t-1}+I_{t}$

**Implication of profit maximization:** From FOCs

$$
\begin{aligned}
r_{t}^{k} & = F_{K}(K_{t-1},L_{t})=A_t \alpha K_{t-1}^{-\theta-1}Y_{t}^{-1}\\
w_{t} & = F_{L}(K_{t-1},L_{t})=A_t (1-\alpha)L_{t}^{-\theta-1}Y_{t}^{-1}
\end{aligned}
$$

**Implication of utility maximization:** From FOCs

$$
\begin{aligned}
C_{t}^{-\sigma}&=\beta(1+r_{t+1})C_{t+1}^{-\sigma} \\
&=\beta(1+F_{K}(K_{t},1)-\delta)C_{t+1}^{-\sigma}
\end{aligned}
$$

**Simpler capital accumulation equation:** 

$$
\begin{aligned}
K_{t} &= (1-\delta)K_{t-1}+I_{t} \\
&= (1-\delta)K_{t-1}+Y_{t}-C_{t} \\
&= (1-\delta)K_{t-1} + F(K_{t-1},1)-C_{t}
\end{aligned}
$$

## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

**Solution algorithm:** 

We can summarize the model in the **non-linear equation system**

$$
\begin{aligned}
\boldsymbol{H}(\boldsymbol{K},\boldsymbol{C},K_{-1})=\left[\begin{array}{c}
H_{0}\\
H_{1}\\
\begin{array}{c}
\vdots\end{array}
\end{array}\right]=\left[\begin{array}{c}
0\\
0\\
\begin{array}{c}
\vdots\end{array}
\end{array}\right]
\end{aligned}
$$

where $\boldsymbol{K} = [K_0,K_1\dots]$, $\boldsymbol{C} = [C_0,C_1\dots]$, and

$$
\begin{aligned}
H_{t}
=\left[\begin{array}{c}
C_{t}^{-\sigma}-\beta(1+F_{K}(K_{t},1))C_{t+1}^{-\sigma}\\
K_{t}-[(1-\delta)K_{t-1} + F(K_{t-1},1)-C_{t}]
\end{array}\right]
\end{aligned}
$$

**Note:** We can verify all equilibrium conditions are fulfilled.

**Path:** We refer to $\boldsymbol{K}$ and $\boldsymbol{C}$ as *transition paths*.

**Implementation:** Assume all variables are in steady  state after some **truncation horizon**.<br>We solve equation system above in **two steps**:

1. Calculate the numerical **jacobian** of $\boldsymbol{H}$ wrt. $\boldsymbol{K}$
and $\boldsymbol{C}$ around the steady state
2. Solve the equation system using a **hand-written Broyden-solver**

**Note:** The equation system can also be solved directly using `scipy.optimize.root`.

**Remember:** The jacobian is just a gradient. I.e. the matrix of what the implied errors are in $\boldsymbol{H}$ when a *single* $K_t$ or $C_t$ change.

In [ ]:
model = RamseyModelClass()
par = model.par
ss = model.ss
path = model.path

**Find steady state:** 

1. Target steady-state capital-output ratio, $K_{ss}/Y_{ss}$ of 4.0.
2. Force steady-state output $Y_{ss} = 1$.
3. Adjust $\beta$ and $\Gamma_{ss}$ to achieve this.

In [ ]:
model.find_steady_state(KY_ss=4.0)

**Test that errors and the path are 0:**

In [ ]:
# a. set initial value
par.K_lag_ini = ss.K

# b. set path
path.Gamma[:] = ss.Gamma
path.C[:] = ss.C
path.K[:] = ss.K

# c. check errors
errors_ss = model.evaluate_path_errors()
assert np.allclose(errors_ss,0.0)

In [ ]:
model.calculate_jacobian()

**Solve:**

In [ ]:
par.K_lag_ini = 0.50*ss.K # start away from steady state
model.solve() # find transition path

In [ ]:
fig = plt.figure(figsize=(6,6/1.5))
ax = fig.add_subplot(1,1,1)
ax.plot(path.K_lag,label=r'$K_{t-1}$')
ax.legend(frameon=True)
fig.tight_layout()

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

**Note:** scipy computes the jacobian internally

In [ ]:
model_scipy = RamseyModelClass()
model_scipy.par.solver = 'scipy'
model_scipy.find_steady_state(KY_ss=4.0)
model_scipy.par.K_lag_ini = 0.50*model_scipy.ss.K
model_scipy.path.Gamma[:] = model_scipy.ss.Gamma
model_scipy.solve()

In [ ]:
fig = plt.figure(figsize=(6,6/1.5))
ax = fig.add_subplot(1,1,1)
ax.plot(path.K_lag,label=r'$K_{t-1}$, broyden')
ax.plot(model_scipy.path.K_lag,ls='--',label=r'$K_{t-1}$, scipy')
ax.legend(frameon=True)
fig.tight_layout()

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

**Persistent technology shock:**

In [ ]:
par.K_lag_ini = ss.K # start from steady state
path.Gamma[:] = 0.95**np.arange(par.Tpath)*0.1*ss.Gamma + ss.Gamma # shock path

**Terminology:** This is called an MIT-shock. Households do not expect shocks.<br>
Know the full path of the shock when it arrives. Continue to believe no future shocks will happen.

**Solve:**

In [ ]:
model.solve()

In [ ]:
fig = plt.figure(figsize=(2*6,6/1.5))

ax = fig.add_subplot(1,2,1)
ax.set_title('Capital, $K_{t-1}$')
ax.plot(path.K_lag)

ax = fig.add_subplot(1,2,2)
ax.plot(path.Gamma)
ax.set_title('Technology, $\Gamma_t$')

fig.tight_layout()

**Future persistent technology shock: Shock happing after period $J$:**

In [ ]:
par.K_lag_ini = ss.K # start from steady state

# shock
J = 50
path.Gamma[:] = ss.Gamma
path.Gamma[J:] = 0.95**np.arange(par.Tpath-J)*0.1*ss.Gamma + ss.Gamma

**Solve:**

In [ ]:
model.solve()

In [ ]:
fig = plt.figure(figsize=(2*6,6/1.5))

ax = fig.add_subplot(1,2,1)
ax.set_title('Capital, $K_{t-1}$')
ax.plot(path.K_lag)

ax = fig.add_subplot(1,2,2)
ax.plot(path.Gamma)
ax.set_title('Technology, $\Gamma_t$')

fig.tight_layout()

# Conclusion

Add concise conclusion. 